# Retry Query Engine

In [ ]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from llama_index.indices.vector_store.base import VectorStoreIndex
from llama_index.query_engine.retry_query_engine import (
    RetryQueryEngine,
    RetryGuidelineQueryEngine,
)
from llama_index.query_engine.retry_source_query_engine import RetrySourceQueryEngine
from llama_index.readers.file.base import SimpleDirectoryReader
from llama_index.evaluation.base import QueryResponseEvaluator
from llama_index.evaluation.guideline_eval import GuidelineEvaluator
from llama_index.response.schema import Response
from llama_index.indices.query.query_transform.feedback_transform import (
    FeedbackQueryTransformation,
)

In [ ]:
uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()
index = VectorStoreIndex.from_documents(uber_docs)
query = "How many drivers does Uber have?"

In [ ]:
# Default query engine
default_query_engine = index.as_query_engine()
response = default_query_engine.query(query)
print(f"Default query engine response: {response}")

In [ ]:
# Query engine with retry
query_response_evaluator = QueryResponseEvaluator()
retry_query_engine = RetryQueryEngine(index.as_query_engine(), query_response_evaluator)
retry_response = retry_query_engine.query(query)
print(f"Query engine with retry response: {retry_response}")

In [ ]:
retry_source_query_engine = RetrySourceQueryEngine(index.as_query_engine(), query_response_evaluator)
retry_source_response = retry_source_query_engine.query(query)
print(f"Query engine with retry source response: {retry_source_response}")

In [3]:

# Guideline eval
guideline_eval = GuidelineEvaluator()
typed_response = response if isinstance(response, Response) else response.get_response()
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval}")
feedback_query_transform = FeedbackQueryTransformation(evaluation=eval, resynthesize_query=True)
transformed_query = feedback_query_transform.run(query)
print(f"Transformed query: {transformed_query}")
retry_guideline_query_engine = RetryGuidelineQueryEngine(index.as_query_engine(), guideline_eval, resynthesize_query=True)
retry_guideline_response = retry_guideline_query_engine.query(query)
print(f"Query engine with retry guideline response: {retry_guideline_response}")

SyntaxError: EOL while scanning string literal (1295256350.py, line 8)